In [27]:
#Import all the necessary modules
%matplotlib inline
import numpy as np
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
#importing seaborn for statistical plots
import seaborn as sns
# Import Linear Regression machine learning library
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
import statsmodels.formula.api as smf
from scipy.stats import zscore
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

### define Function for prediction and regression results 

In [28]:
# Predict the model and return the train-score, test-score and rmse-value
def customized_model(model, xtrain, ytrain, xtest, ytest): 
    m1.fit(xtrain, ytrain)

    # Predict the Test-Data
    ypred = m1.predict(xtest)

    # Calculate the RMSE Score
    r2_score = m1.score(xtest, ytest)
    rmse_test = np.sqrt(np.mean((y_test-ypred)**2))

    result = [r2_score, rmse_test]
    return result 

In [29]:
# 1.Select a dataset from the repository, select a suitable algorithm to evaluate and use the holdout method to estimate the future performance.
#Import data from dataset
data = pd.read_csv('kin-32nm.csv')

In [30]:
data.head()

,theta1,theta2,theta3,theta4,theta5,theta6,theta7,theta8,alpha1,alpha2,...,a8,d1,d2,d3,d4,d5,d6,d7,d8,y
0,-0.313332,-0.867431,1.216446,-0.685140,-1.214539,1.314975,-0.315100,0.838499,-0.785398,-0.785398,...,0.000000,0.342542,0.000000,0.261032,0.000000,0.232554,0.392556,0.483856,0.000000,1.708010
1,0.295689,-0.357731,0.622438,-0.186873,0.816770,-1.302975,0.945227,-0.597122,1.570796,0.000000,...,0.264764,0.000000,0.371554,0.336131,0.394898,0.000000,0.000000,0.449238,0.262860,1.946530
2,-1.248976,0.585276,0.400678,-1.372419,-0.668149,-1.536149,-0.447459,0.533924,1.570796,-1.570796,...,0.216391,0.000000,0.000000,0.184938,0.000000,0.000000,0.000000,0.000000,0.267063,1.434310
3,-0.727581,-1.377569,0.727146,0.354231,0.247096,-0.249870,0.746423,0.747372,1.570796,-1.570796,...,0.000000,0.446133,0.244500,0.000000,0.295639,0.399661,0.000000,0.272555,0.000000,1.837355
4,0.092724,0.185064,0.186382,0.362497,0.932209,0.518644,-1.075121,0.879169,1.570796,-1.570796,...,0.353262,0.000000,0.050852,0.377702,0.193091,0.225181,0.335084,0.183534,0.283024,1.409236


In [31]:
# Split the data into X Features and y Target 
X =  data.drop("y", axis=1)
y =  data.y

In [32]:
X = X.apply(zscore)

In [33]:
#Split data into training and test set in the ratio of 70 and 30 using hold-out method
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=1)
ResultsDf =  pd.DataFrame( index=["R2-Score", "RMSE-Score"])


### Linear Regression 

In [34]:
# Use Linear Regression Model for training
model1 = LinearRegression()
ResultsDf['Linear-Regression'] = customized_model(model1, X_train, y_train, X_test, y_test)
ResultsDf

,Linear-Regression
R2-Score,0.274934
RMSE-Score,0.387324


# 2. Select a dataset from the repository, select a suitable algorithm to evaluate and use cross validation to estimate the future performance.

### Common Function for reading-data, train-test split, model-processing, kfold cross-validation and calculating r2-score

In [59]:
from sklearn.model_selection import KFold,cross_val_score
def customized_model_processing(data_file, model_lst):
    #Import data from dataset
    data = pd.read_csv(data_file)
    
    # Split the data into X Features and y Target 
    y =  data.y
    X =  data.drop(columns = ['y'])
    
    
    X = X.apply(zscore)
    
    # evaluate each model with kfold cross-validation using r2-score 
    #ResultsDf =  pd.DataFrame( index=['mean r2-score', 'r2-score variance'])
    result_dict = {}
    
    for name, model in model_lst:
        kfold = KFold(n_splits=3,random_state=2)
        cv_results = cross_val_score(model, X, y, scoring='r2', cv=kfold)
        #results = []
        #results.append(np.mean(cv_results))
        #results.append(np.var(cv_results))
        #ResultsDf[name] = results
        result_dict[name] = np.mean(cv_results)
    return result_dict

#### Use Simple Linear Regression Model for training

In [60]:
model2 = LinearRegression()
models = []
models.append(('Simple-Linear-Regression Mean-R2-Score',model2))

#### Run the linear-regression on the same data-set as used in (1)

In [61]:
result_model2_crossval = customized_model_processing('kin-32nm.csv', models)
result_model2_crossval

{'Simple-Linear-Regression Mean-R2-Score': 0.26304607849447914}

# 3. Select a dataset from the repository and compare the performance of two different algorithms on the same dataset. 

### Use LinearRegression and RandomForestRegressor to evaluate on new dataset and compare results 

In [62]:
from sklearn.ensemble import RandomForestRegressor

model2 = LinearRegression()
model3 = RandomForestRegressor(max_depth=10, n_estimators=20, random_state=7, verbose=False)

In [63]:
models = []
models.append(('Simple-Linear-Regression Mean-R2-Score',model2))
models.append(('Random-Forest-Regression Mean-R2-Score', model3))

In [64]:
data.y

0       1.708010
1       1.946530
2       1.434310
3       1.837355
4       1.409236
          ...   
8187    0.807823
8188    1.389872
8189    1.456773
8190    1.425701
8191    1.523805
Name: y, Length: 8192, dtype: float64

In [65]:
results_df = customized_model_processing('pumadyn-32nm.csv', models)
results_df

AttributeError: 'DataFrame' object has no attribute 'y'

# 4. Select at least 10 different datasets from the repository and compare the performance of at two different algorithms on all 10 datasets

In [26]:
import glob
count = 0
ResultsDf =  pd.DataFrame( index=['Linear-Regression Mean-R2-Score', 'Random-Forest-Regression Mean-R2-Score'])

for filename in glob.glob('*.csv'):
    #print(filename)
    if (count >=10):
        break
    count = count + 1    
    results_dict = customized_model_processing(filename, models)
    ResultsDf[filename] = [results_dict['Simple-Linear-Regression Mean-R2-Score'], results_dict['Random-Forest-Regression Mean-R2-Score']]
    #print (results_dict)

KeyError: "['y'] not found in axis"

In [252]:
ResultsDf.T

,Linear-Regression Mean-R2-Score,Random-Forest-Regression Mean-R2-Score
.\Regression\kin-32fh.csv,0.411236,0.645484
.\Regression\kin-32fm.csv,0.905189,0.667558
.\Regression\kin-32nh.csv,0.204904,0.150486
.\Regression\kin-32nm.csv,0.263046,0.203796
.\Regression\kin-8fh.csv,0.716412,0.687158
.\Regression\kin-8fm.csv,0.919725,0.889537
.\Regression\kin-8nh.csv,0.359587,0.479951
.\Regression\kin-8nm.csv,0.411236,0.645484
.\Regression\pumadyn-32fh.csv,0.431363,0.423655
.\Regression\pumadyn-32fm.csv,0.929547,0.962204
